In [1]:
# import os
import pandas as pd
import ollama as o

# Data preprocess

In [2]:
styles2 = pd.read_csv('/Users/nkapila6/Downloads/styles2.csv', index_col=0).dropna()
styles2.head(3)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch


In [3]:
cols = ['gender', 'masterCategory', 'subCategory', 'articleType',
       'baseColour', 'season', 'usage', 'productDisplayName']

In [4]:
imgs = pd.read_csv('/Users/nkapila6/Downloads/images.csv')
imgs["id"] = imgs.filename.str[0:-4]
imgs.head(3)

,filename,link,id
0,15970.jpg,http://assets.myntassets.com/v1/images/style/p...,15970
1,39386.jpg,http://assets.myntassets.com/v1/images/style/p...,39386
2,59263.jpg,http://assets.myntassets.com/v1/images/style/p...,59263


In [5]:
# merging all the tags for a more "rich" embedding, double quotes intended
styles2["desc"] = styles2[cols].apply(lambda x: " ".join(x), axis=1)
styles2.head(3)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,desc
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,Men Apparel Topwear Shirts Navy Blue Fall Casu...
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,Men Apparel Bottomwear Jeans Blue Summer Casua...
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,Women Accessories Watches Watches Silver Winte...


In [6]:
imgs.id = imgs.id.astype(int)
print(len(styles2), len(imgs))

44079 44446


In [7]:
# merging image links with description, only ones that exist :) 
data = pd.merge(imgs, styles2[["id", "desc"]], on="id") 
data.to_csv('preliminary_data.csv')
data

,filename,link,id,desc
0,15970.jpg,http://assets.myntassets.com/v1/images/style/p...,15970,Men Apparel Topwear Shirts Navy Blue Fall Casu...
1,39386.jpg,http://assets.myntassets.com/v1/images/style/p...,39386,Men Apparel Bottomwear Jeans Blue Summer Casua...
2,59263.jpg,http://assets.myntassets.com/v1/images/style/p...,59263,Women Accessories Watches Watches Silver Winte...
3,21379.jpg,http://assets.myntassets.com/v1/images/style/p...,21379,Men Apparel Bottomwear Track Pants Black Fall ...
4,53759.jpg,http://assets.myntassets.com/v1/images/style/p...,53759,Men Apparel Topwear Tshirts Grey Summer Casual...
...,...,...,...,...
44074,17036.jpg,http://assets.myntassets.com/v1/images/style/p...,17036,Men Footwear Shoes Casual Shoes White Summer C...
44075,6461.jpg,http://assets.myntassets.com/v1/images/style/p...,6461,Men Footwear Flip Flops Flip Flops Red Summer ...
44076,18842.jpg,http://assets.myntassets.com/v1/images/style/p...,18842,Men Apparel Topwear Tshirts Blue Fall Casual P...
44077,46694.jpg,http://assets.myntassets.com/v1/images/style/p...,46694,Women Personal Care Fragrance Perfume and Body...


# Embedding extraction

In [8]:
o.embeddings?

Signature:
o.embeddings(
    model: str = '',
    prompt: Optional[str] = None,
    options: Union[Mapping[str, Any], ollama._types.Options, NoneType] = None,
    keep_alive: Union[float, str, NoneType] = None,
) -> ollama._types.EmbeddingsResponse
Docstring: Deprecated in favor of `embed`.
File:      ~/Code/mcp-sticky/.venv/lib/python3.10/site-packages/ollama/_client.py
Type:      method

In [9]:
o.embed?

Signature:
o.embed(
    model: str = '',
    input: Union[str, Sequence[str]] = '',
    truncate: Optional[bool] = None,
    options: Union[Mapping[str, Any], ollama._types.Options, NoneType] = None,
    keep_alive: Union[float, str, NoneType] = None,
) -> ollama._types.EmbedResponse
Docstring: <no docstring>
File:      ~/Code/mcp-sticky/.venv/lib/python3.10/site-packages/ollama/_client.py
Type:      method

In [10]:
from tqdm import tqdm

def get_embedding(text)->list:
        resp = o.embed(model='nomic-embed-text', input=text)
        return resp.embeddings[0]

tqdm.pandas(desc="Embedding texts");
data["embedding_model"] = "nomic-embed-text"
data["vector"] = data["desc"].progress_apply(get_embedding)

Embedding texts: 100%|██████████| 44079/44079 [16:50<00:00, 43.63it/s]


In [13]:
data.head(3)

,filename,link,id,desc,embedding_model,vector
0,15970.jpg,http://assets.myntassets.com/v1/images/style/p...,15970,Men Apparel Topwear Shirts Navy Blue Fall Casu...,nomic-embed-text,"[-0.036835346, -0.004350252, -0.15560359, -0.0..."
1,39386.jpg,http://assets.myntassets.com/v1/images/style/p...,39386,Men Apparel Bottomwear Jeans Blue Summer Casua...,nomic-embed-text,"[-0.053665683, -0.02066088, -0.14330265, -0.02..."
2,59263.jpg,http://assets.myntassets.com/v1/images/style/p...,59263,Women Accessories Watches Watches Silver Winte...,nomic-embed-text,"[0.038639385, 0.0112380795, -0.18502164, 0.051..."


In [12]:
data.to_csv('full_data.csv')

In [22]:
# limiting to 3k examples (couchbase limitation)
data.head(3000).to_csv('full_data_3k.csv', index=False)

In [23]:
import json

with open('full_data_3k.jsonl', 'w') as f:
    for _, row in data.head(3000).iterrows():
        json.dump(row.to_dict(), f)
        f.write('\n')